# **Random Forest model**

# Setup

Primero se debe instalar la libreria **KerasTuner** con `pip install keras-tuner -q` e importarla de la siguiente como `import keras_tuner`

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from joblib import dump
from sklearn import metrics
from tensorflow import keras
import keras_tuner
import glob

# Importar datos de entrenamiento

In [2]:
X_train = pd.read_csv('X_test.csv').to_numpy()
y_train = pd.read_csv('y_test.csv').to_numpy()
y_train = np.ravel(y_train)

# Construccion del modelo

Definimos una funcion que reciba el argumento `hp` y la configuracion de hiperparametros como se muestra a continuacion donde la funcion `Int` indica que un hiperparámetro es un número entero como el numero de estimadores `n_estimators` y profundidad maxima `max_depth`.

In [4]:
def build_model(hp):
    n_estimators = hp.Int("n_estimators", 100, 1000, 100)
    max_depth = hp.Int("max_depth", 10, 100, 10)
    model = RandomForestClassifier(
        n_estimators = n_estimators,
        max_depth = max_depth
    )
    return model

# Evaluacion y seleccion del modelo

## Evaluacion del modelo (*k*-fold-cross-validation)

Definimos la funcion `StratifiedKFold` para generar la validacion cruzada y evaluar la capacidad de generalizacion del modelo.

In [5]:
cv = StratifiedKFold(n_splits = 10, random_state = None, shuffle = False)

## Seleccion del modelo (Bayesian Optimization)

Definimos la funcion `SklearnTuner` y el metodo seleccion de parametros `BayesianOptimizationOracle` que es recibida como un argumento de la funcion de ajuste de hiperparametros.

In [6]:
tuner = keras_tuner.tuners.SklearnTuner(
    oracle = keras_tuner.oracles.BayesianOptimizationOracle(
        objective = keras_tuner.Objective('score', 'max'),
        max_trials = 10
    ),
    hypermodel = build_model,
    scoring = metrics.make_scorer(metrics.accuracy_score),
    cv = cv,
    directory = 'SklearnTunerResults',
    project_name='RF_training',
    overwrite = True
    )

Iniciamos la busqueda de hiperparametros con funcion `search`.

In [7]:
tuner.search(X_train, y_train)

Trial 10 Complete [00h 00m 01s]
score: 0.9111111111111112

Best score So Far: 0.9111111111111112
Total elapsed time: 00h 00m 38s
INFO:tensorflow:Oracle triggered exit


Obtenemos el mejor modelo con la funcion `get_best_models`.

In [8]:
best_model = tuner.get_best_models(num_models = 1)[0]

# Guardar el modelo

In [9]:
dump(best_model, 'RF_model.joblib')

['RF_model.joblib']